In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import random
random.seed(42)

In [ ]:
def HolidayFeatureCreator(input_data):
    """
    Receives as input a DataFrame or Series and creates a column named "Holidays_flag" with 1 if there is a holiday and with 0 if no holidays exist.
    Holidays derived from Germany.
    """
    if isinstance(input_data, pd.Series):
        input_df = pd.DataFrame(input_data)
    elif isinstance(input_data, pd.DataFrame):
        input_df = input_data
    else:
        raise ValueError("Input must be a DataFrame or Series.")

    assert isinstance(input_df.index, pd.DatetimeIndex), "Index must be a datetime index."

    national_holidays_all = holidays.US(years=[2014,2015,2016,2017,2018,2019,2020, 2021, 2022, 2023, 2024, 2025, 2026]).items()
    national_holidays = [items[0] for items in national_holidays_all]  # this is a list

    # Create a new column for holidays flag
    input_df["Holidays_flag"] = 0

    # Iterate over the index and set holiday flag to 1 if the date matches any national holiday
    for index_date in input_df.index:
        if index_date.date() in national_holidays:
            input_df.at[index_date, "Holidays_flag"] = 1

    return input_df

In [ ]:
pip install holidays

In [ ]:
pip install "flaml[automl]"


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import math
import holidays
from flaml import AutoML
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, classification_report


# run before

In [ ]:
encryption_level=["10","20","30","40","50","60","70","80","90","100"]
type_of_encryption=["random","strategy"]
for g in type_of_encryption:
  print(g)
  for i in encryption_level:
    print(i)
    if g=="random":
      FileName1=f"modified_{g}shuffle_{i}_key.csv"
    if g=="strategy":
      FileName1=f"encrypted_dataframe_with_Key_{i}percent_{g}.csv"

    print(FileName1)
    Folder_Path1="/content/gdrive/MyDrive/Encryption paper" #path of the folder
    FilePath1=os.path.join(Folder_Path1,FileName1)

    df = pd.read_csv(FilePath1, index_col=0,sep=",", parse_dates=[0], engine="pyarrow")

    FileName="original_df.csv"
    Folder_Path="/content/gdrive/MyDrive/Encryption paper" #path of the folder
    FilePath=os.path.join(Folder_Path,FileName)


    df_no_encryption = pd.read_csv(FilePath, index_col=0,sep=",", parse_dates=[0], engine="pyarrow")

    # Sort the dataframe by its index in ascending order
    df = df.sort_index(ascending=True)
    df_no_encryption = df_no_encryption.sort_index(ascending=True)

    df_no_encryption=df_no_encryption[["Total_load","T_LR","RH_LR","TotalOccupancy","HP_load"]]

    df_no_encryption=HolidayFeatureCreator(df_no_encryption)
    df_no_encryption["minute"]=df_no_encryption.index.minute
    df_no_encryption["hour"]= df_no_encryption.index.hour
    df_no_encryption["sin_hour"]= np.sin(df_no_encryption.index.hour*2*np.pi/24) #2πh/24 the hour 23 and 0 are close to one another but we want to have period function but sin of 23 and sin 0 are way apart one another
    df_no_encryption["cos_hour"]= np.cos(df_no_encryption.index.hour*2*np.pi/24)
    df_no_encryption["dayOfWeek"]=df_no_encryption.index.dayofweek
    df_no_encryption["month"]=df_no_encryption.index.month
    df_no_encryption.loc[df_no_encryption["dayOfWeek"]>4, "weekendFlag"]=1 #when day of the week is larger than 4 (5,6) put weekend flag to be 1
    df_no_encryption.loc[df_no_encryption["dayOfWeek"]<5, "weekendFlag"]=0 #less that 5 is (4,3,2,1,0) 0 is work day, 1 is weekend
    df_no_encryption.loc[:,"workingHour"]=0
    df_no_encryption.loc[((df_no_encryption["hour"]>8) & (df_no_encryption["hour"]<19)), "workingHour"]=1

    df=df[["Total_load","T_LR","RH_LR","TotalOccupancy","HP_load"]]
    df=HolidayFeatureCreator(df)
    df["minute"]=df.index.minute
    df["hour"]= df.index.hour
    df["sin_hour"]= np.sin(df.index.hour*2*np.pi/24) #2πh/24 the hour 23 and 0 are close to one another but we want to have period function but sin of 23 and sin 0 are way apart one another
    df["cos_hour"]= np.cos(df.index.hour*2*np.pi/24)
    df["dayOfWeek"]=df.index.dayofweek
    df["month"]=df.index.month
    df.loc[df["dayOfWeek"]>4, "weekendFlag"]=1 #when day of the week is larger than 4 (5,6) put weekend flag to be 1
    df.loc[df["dayOfWeek"]<5, "weekendFlag"]=0 #less that 5 is (4,3,2,1,0) 0 is work day, 1 is weekend
    df.loc[:,"workingHour"]=0
    df.loc[((df["hour"]>8) & (df["hour"]<19)), "workingHour"]=1

    # Prepare the dataset
    df['TotalOccupancy'] = df['TotalOccupancy'].astype(int)  # Ensure target column is integer
    target = 'TotalOccupancy'
    features = df.drop(columns=[target]).columns

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.2, shuffle=False,random_state=42)
    X_train_no_encryption, X_test_no_encryption, y_train_no_encryption, y_test_no_encryption = train_test_split(df_no_encryption[features], df_no_encryption[target], test_size=0.2, shuffle=False,random_state=42)

    # Columns to add noise to
    columns_to_add_noise = ['Total_load', 'T_LR', 'RH_LR', 'HP_load']
    noise_std_fraction = 0.1  # 5% of standard deviation

    # Add noise to each specified column
    for col in columns_to_add_noise:
        std_dev = X_test[col].std()  # Compute the standard deviation of the column
        noise = np.random.normal(loc=0, scale=noise_std_fraction * std_dev, size=X_test[col].shape)
        X_test[col] = X_test[col] + noise

    # FLAML AutoML setup
    automl = AutoML()
    settings = {
        "time_budget": 60,  # Total time in seconds for optimization
        "metric": "accuracy",  # Metric for evaluation
        "task": "classification",  # Task type
        "log_file_name": "flaml_classification.log",  # Log file name
        "split_type": "stratified",  # Use cross-validation
        "n_splits": 5,  # Number of folds for cross-validation
    }

    # Train the model
    automl.fit(X_train=X_train, y_train=y_train, **settings)

    # Evaluate the model
    y_pred = automl.predict(X_test)
    print("Classification Report:\n", classification_report(y_test_no_encryption, y_pred))

    # Confusion Matrix
    cm = confusion_matrix(y_test_no_encryption, y_pred)
    print("Confusion Matrix:\n", cm)

    # Assuming y_test and y_pred are already defined
    accuracy = accuracy_score(y_test_no_encryption, y_pred)
    recall = recall_score(y_test_no_encryption, y_pred, average='binary')  # Use 'binary' for binary classification
    f1 = f1_score(y_test_no_encryption, y_pred, average='binary')  # Use 'binary' for binary classification

    print("Accuracy:", accuracy)
    print("Recall:", recall)
    print("F1-Score:", f1)


    # Calculate metrics for class 0
    precision_0 = precision_score(y_test_no_encryption, y_pred, pos_label=0)
    recall_0 = recall_score(y_test_no_encryption, y_pred, pos_label=0)
    f1_0 = f1_score(y_test_no_encryption, y_pred, pos_label=0)

    # Calculate metrics for class 1
    precision_1 = precision_score(y_test_no_encryption, y_pred, pos_label=1)
    recall_1 = recall_score(y_test_no_encryption, y_pred, pos_label=1)
    f1_1 = f1_score(y_test_no_encryption, y_pred, pos_label=1)

    # Print the metrics
    print("Metrics for Class 0:")
    print("Precision (Class 0):", precision_0)
    print("Recall (Class 0):", recall_0)
    print("F1-Score (Class 0):", f1_0)

    print("\nMetrics for Class 1:")
    print("Precision (Class 1):", precision_1)
    print("Recall (Class 1):", recall_1)
    print("F1-Score (Class 1):", f1_1)

    FileNamemetrics="metrics.csv"
    FilePath=os.path.join(Folder_Path,FileNamemetrics)
    df_metrics=pd.read_csv(FilePath,index_col=0)

    metrics_data = {
        "Accuracy": [accuracy],
        "Recall (Overall)": [recall],
        "F1-Score (Overall)": [f1],
        "Precision (Class 0)": [precision_0],
        "Recall (Class 0)": [recall_0],
        "F1-Score (Class 0)": [f1_0],
        "Precision (Class 1)": [precision_1],
        "Recall (Class 1)": [recall_1],
        "F1-Score (Class 1)": [f1_1],
    }

    df_metrics_new = pd.DataFrame(metrics_data, index=[f"{g}_encryption{i}"])
    df_metrics=pd.concat([df_metrics,df_metrics_new])
    df_metrics.to_csv(FilePath, index=True)


random
10
modified_randomshuffle_10_key.csv
[flaml.automl.logger: 02-04 16:03:59] {1728} INFO - task = classification
[flaml.automl.logger: 02-04 16:03:59] {1739} INFO - Evaluation method: holdout
[flaml.automl.logger: 02-04 16:03:59] {1838} INFO - Minimizing error metric: 1-accuracy
[flaml.automl.logger: 02-04 16:03:59] {1955} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'xgboost', 'extra_tree', 'xgb_limitdepth', 'sgd', 'lrl1']
[flaml.automl.logger: 02-04 16:03:59] {2258} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 02-04 16:03:59] {2393} INFO - Estimated sufficient time budget=810s. Estimated necessary time budget=19s.
[flaml.automl.logger: 02-04 16:03:59] {2442} INFO -  at 0.2s,	estimator lgbm's best error=0.2727,	best estimator lgbm's best error=0.2727
[flaml.automl.logger: 02-04 16:03:59] {2258} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 02-04 16:03:59] {2442} INFO -  at 0.3s,	estimator lgbm's best error=0.2727,	best estimator lgbm

INFO:flaml.tune.searcher.blendsearch:No low-cost partial config given to the search algorithm. For cost-frugal search, consider providing low-cost values for cost-related hps via 'low_cost_partial_config'. More info can be found at https://microsoft.github.io/FLAML/docs/FAQ#about-low_cost_partial_config-in-tune


[flaml.automl.logger: 02-04 16:03:59] {2442} INFO -  at 0.4s,	estimator sgd's best error=0.2727,	best estimator lgbm's best error=0.2727
[flaml.automl.logger: 02-04 16:03:59] {2258} INFO - iteration 3, current learner sgd
[flaml.automl.logger: 02-04 16:04:01] {2442} INFO -  at 1.6s,	estimator sgd's best error=0.2727,	best estimator lgbm's best error=0.2727
[flaml.automl.logger: 02-04 16:04:01] {2258} INFO - iteration 4, current learner lgbm
[flaml.automl.logger: 02-04 16:04:01] {2442} INFO -  at 2.0s,	estimator lgbm's best error=0.0885,	best estimator lgbm's best error=0.0885
[flaml.automl.logger: 02-04 16:04:01] {2258} INFO - iteration 5, current learner xgboost
[flaml.automl.logger: 02-04 16:04:01] {2442} INFO -  at 2.4s,	estimator xgboost's best error=0.2727,	best estimator lgbm's best error=0.0885
[flaml.automl.logger: 02-04 16:04:01] {2258} INFO - iteration 6, current learner extra_tree
[flaml.automl.logger: 02-04 16:04:01] {2442} INFO -  at 2.5s,	estimator extra_tree's best error

INFO:flaml.tune.searcher.blendsearch:No low-cost partial config given to the search algorithm. For cost-frugal search, consider providing low-cost values for cost-related hps via 'low_cost_partial_config'. More info can be found at https://microsoft.github.io/FLAML/docs/FAQ#about-low_cost_partial_config-in-tune


[flaml.automl.logger: 02-04 16:05:01] {2442} INFO -  at 0.3s,	estimator sgd's best error=0.2721,	best estimator lgbm's best error=0.1048
[flaml.automl.logger: 02-04 16:05:01] {2258} INFO - iteration 4, current learner lgbm
[flaml.automl.logger: 02-04 16:05:01] {2442} INFO -  at 0.4s,	estimator lgbm's best error=0.0969,	best estimator lgbm's best error=0.0969
[flaml.automl.logger: 02-04 16:05:01] {2258} INFO - iteration 5, current learner lgbm
[flaml.automl.logger: 02-04 16:05:01] {2442} INFO -  at 0.5s,	estimator lgbm's best error=0.0969,	best estimator lgbm's best error=0.0969
[flaml.automl.logger: 02-04 16:05:01] {2258} INFO - iteration 6, current learner lgbm
[flaml.automl.logger: 02-04 16:05:01] {2442} INFO -  at 0.5s,	estimator lgbm's best error=0.0897,	best estimator lgbm's best error=0.0897
[flaml.automl.logger: 02-04 16:05:01] {2258} INFO - iteration 7, current learner lgbm
[flaml.automl.logger: 02-04 16:05:02] {2442} INFO -  at 0.6s,	estimator lgbm's best error=0.0897,	best es

...

[flaml.automl.logger: 02-04 16:23:23] {2442} INFO -  at 60.6s,	estimator lrl1's best error=0.1607,	best estimator lgbm's best error=0.0006
[flaml.automl.logger: 02-04 16:23:24] {2685} INFO - retrain lgbm for 0.6s
[flaml.automl.logger: 02-04 16:23:24] {2688} INFO - retrained model: LGBMClassifier(colsample_bytree=0.6553390318506341,
               learning_rate=0.07867211164601755, max_bin=255,
               min_child_samples=8, n_estimators=205, n_jobs=-1, num_leaves=33,
               reg_alpha=0.054896482612159424, reg_lambda=0.0019641706878982795,
               verbose=-1)
[flaml.automl.logger: 02-04 16:23:24] {1985} INFO - fit succeeded
[flaml.automl.logger: 02-04 16:23:24] {1986} INFO - Time taken to find the best model: 29.56532859802246
Classification Report:
               precision    recall  f1-score   support

           0       0.36      0.62      0.46      1095
           1       0.82      0.61      0.70      3053

    accuracy                           0.61      4148
  

INFO:flaml.tune.searcher.blendsearch:No low-cost partial config given to the search algorithm. For cost-frugal search, consider providing low-cost values for cost-related hps via 'low_cost_partial_config'. More info can be found at https://microsoft.github.io/FLAML/docs/FAQ#about-low_cost_partial_config-in-tune


[flaml.automl.logger: 02-04 16:23:26] {2442} INFO -  at 0.8s,	estimator sgd's best error=0.3076,	best estimator lgbm's best error=0.0289
[flaml.automl.logger: 02-04 16:23:26] {2258} INFO - iteration 4, current learner lgbm
[flaml.automl.logger: 02-04 16:23:26] {2442} INFO -  at 1.0s,	estimator lgbm's best error=0.0114,	best estimator lgbm's best error=0.0114
[flaml.automl.logger: 02-04 16:23:26] {2258} INFO - iteration 5, current learner lgbm
[flaml.automl.logger: 02-04 16:23:26] {2442} INFO -  at 1.0s,	estimator lgbm's best error=0.0114,	best estimator lgbm's best error=0.0114
[flaml.automl.logger: 02-04 16:23:26] {2258} INFO - iteration 6, current learner lgbm
[flaml.automl.logger: 02-04 16:23:26] {2442} INFO -  at 1.1s,	estimator lgbm's best error=0.0072,	best estimator lgbm's best error=0.0072
[flaml.automl.logger: 02-04 16:23:26] {2258} INFO - iteration 7, current learner xgboost
[flaml.automl.logger: 02-04 16:23:26] {2442} INFO -  at 1.2s,	estimator xgboost's best error=0.0560,	b

INFO:flaml.tune.searcher.blendsearch:No low-cost partial config given to the search algorithm. For cost-frugal search, consider providing low-cost values for cost-related hps via 'low_cost_partial_config'. More info can be found at https://microsoft.github.io/FLAML/docs/FAQ#about-low_cost_partial_config-in-tune


[flaml.automl.logger: 02-04 16:24:26] {2442} INFO -  at 61.1s,	estimator lrl1's best error=0.1638,	best estimator xgboost's best error=0.0000
[flaml.automl.logger: 02-04 16:24:28] {2685} INFO - retrain xgboost for 2.0s
[flaml.automl.logger: 02-04 16:24:28] {2688} INFO - retrained model: XGBClassifier(base_score=None, booster=None, callbacks=[],
              colsample_bylevel=0.6592543691287364, colsample_bynode=None,
              colsample_bytree=0.6852937865602421, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=None,
              grow_policy='lossguide', importance_type=None,
              interaction_constraints=None, learning_rate=1.0, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=0, max_leaves=165,
              min_child_weight=0.2941125106838653, missing=nan,
              monotone_constraints=None, multi_strateg